# AdventureWorks – Försäljningsanalys

Denna notebook innehåller en analys av försäljningsdata från AdventureWorks-databasen.
Data hämtas via SQL och analyseras vidare i Python med Pandas och visualiseras med Matplotlib.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

server = "localhost,1433"
database = "AdventureWorks2025"
username = "sa"
password = quote_plus("Str0ngPass!2025")
driver = quote_plus("ODBC Driver 18 for SQL Server")

connection_string = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    f"?driver={driver}&Encrypt=yes&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)

In [ ]:
with engine.connect() as conn:
    print(conn.execute(text("SELECT DB_NAME()")).scalar())

In [ ]:
def read_sql_df(query: str) -> pd.DataFrame:
    with engine.connect() as conn:
        return pd.read_sql(text(query), conn)

## 1. Antal produkter per kategori

In [ ]:
query_1 = """
SELECT
    pc.Name AS Category,
    COUNT(DISTINCT p.ProductID) AS ProductCount
FROM Production.ProductCategory pc
JOIN Production.ProductSubcategory psc
    ON psc.ProductCategoryID = pc.ProductCategoryID
JOIN Production.Product p
    ON p.ProductSubcategoryID = psc.ProductSubcategoryID
GROUP BY pc.Name
ORDER BY ProductCount DESC;
"""

df_1 = read_sql_df(query_1)
df_1

In [ ]:
plt.figure()
plt.bar(df_1["Category"], df_1["ProductCount"])
plt.title("Antal produkter per kategori")
plt.xlabel("Kategori")
plt.ylabel("Antal produkter")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [ ]:
most = df_1.iloc[0]
least = df_1.iloc[-1]
most["Category"], int(most["ProductCount"]), least["Category"], int(least["ProductCount"])


**Insikt:** Kategorien med flest produkter är **Bikes** med **99** produkter.
Kategorien med minst produkter är **Clothing** med **25** produkter.